# Capstone Case Study: Cascade Reservoir Optimization

This notebook serves as a capstone project, bringing together many of the concepts and models we've documented in previous tutorials. We will tackle a realistic and complex problem: **the optimized scheduling of a cascade reservoir system**.

This example is based on the case study described in `chs-knowledge-hub/docs/case_studies/cascade_reservoir_optimization.md`, but provides a complete, runnable implementation using the `SimulationManager`.

## 1. Problem Definition

We have a system of two reservoirs in series (a cascade system). The outflow from the upstream reservoir becomes the inflow for the downstream reservoir.

**Goal:** Maximize the total energy generated by both reservoirs over a 24-hour period.

**Challenge:** Simply releasing as much water as possible from both reservoirs might generate a lot of power, but it could also drain the reservoirs, leaving no water for future use, or violate water level safety constraints. The goal is to find a *schedule* of releases that produces the most power while ending the day in a good state.

## 2. System Setup (YAML Configuration)

We will define our entire system topology in a single YAML configuration. This is the standard way to use the CHS SDK for complex problems. We will use the `NonlinearTank` for our reservoirs to make it more realistic.

In [ ]:
import yaml

cascade_config_yaml = """
simulation_params:
  total_time: 86400  # 24 hours in seconds
  dt: 3600         # 1-hour time step

components:
  upstream_inflow:
    type: TimeSeriesDisturbance
    params:
      times: [0, 86400]
      values: [200, 200] # Constant natural inflow of 200 m^3/s
  
  downstream_inflow:
    type: TimeSeriesDisturbance
    params:
      times: [0, 86400]
      values: [50, 50] # Constant local inflow of 50 m^3/s
      
  res_1: # Upstream Reservoir
    type: NonlinearTank
    params:
      initial_level: 195.0
      min_level: 180.0
      max_level: 200.0
      level_to_volume:
        # Using a direct numpy array representation in YAML
        !numpy.array
        - - [180.0, 190.0, 200.0] # levels
        - - [4.0e8, 4.8e8, 6.0e8] # volumes

  channel_12:
    type: MuskingumChannelModel
    params:
      K: 2.0 # 2-hour travel time
      x: 0.2
      dt: 3600
      initial_inflow: 0.0
      initial_outflow: 0.0

  res_2: # Downstream Reservoir
    type: NonlinearTank
    params:
      initial_level: 145.0
      min_level: 135.0
      max_level: 150.0
      level_to_volume:
        !numpy.array
        - - [135.0, 140.0, 150.0]
        - - [2.0e8, 2.7e8, 4.0e8]

connections: [] # Connections will be handled by the optimization loop

execution_order: [] # Execution will be handled by the optimization loop

logger_config:
  - res_1.state.level
  - res_1.input.release_outflow
  - res_2.state.level
  - res_2.input.release_outflow
"""

# Custom constructor to handle !numpy.array tag
def numpy_constructor(loader, tag_suffix, node):
    return np.array(loader.construct_sequence(node))

yaml.add_multi_constructor('!numpy.array', numpy_constructor, Loader=yaml.SafeLoader)

config = yaml.safe_load(cascade_config_yaml)
print("YAML configuration loaded successfully.")

## 3. Optimization Strategy

A full optimization would use a dedicated solver (like `scipy.optimize`). For this tutorial, we will use a simplified but effective strategy: a **heuristic search**. We will test a few predefined release strategies and choose the one that produces the most power.

Our strategies will be:
1.  **Steady**: Release a constant amount of water, just enough to balance the average inflow.
2.  **Peak Hours**: Release more water during peak electricity demand hours and less during off-peak hours.
3.  **Drain Down**: Release a large amount of water initially, draining the reservoir.

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy

# Add the project root to the path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from water_system_sdk.src.chs_sdk.simulation_manager import SimulationManager

# Define a simple power calculation function
def calculate_power(outflow, head):
    # Power (MW) = g * rho * Q * H * eff
    # We simplify by assuming g*rho*eff is a constant factor
    return outflow * head * 0.0085 # Simplified factor for MW

# This function runs one full 24-hour simulation for a given release schedule
def run_simulation_with_schedule(base_config, schedule):
    sim_config = copy.deepcopy(base_config)
    manager = SimulationManager(sim_config)
    
    history = []
    total_power = 0
    n_steps = int(sim_config['simulation_params']['total_time'] / sim_config['simulation_params']['dt'])
    
    # Get component handles
    res1 = manager.components['res_1']
    res2 = manager.components['res_2']
    channel = manager.components['channel_12']
    up_inflow = manager.components['upstream_inflow']
    down_inflow = manager.components['downstream_inflow']
    
    for i in range(n_steps):
        # 1. Set releases based on the schedule
        res1.input.release_outflow = schedule['res_1'][i]
        res2.input.release_outflow = schedule['res_2'][i]
        
        # 2. Run disturbances
        up_inflow.step(dt=manager.config.simulation_params.dt, t=i*manager.config.simulation_params.dt)
        down_inflow.step(dt=manager.config.simulation_params.dt, t=i*manager.config.simulation_params.dt)
        
        # 3. Update reservoir 1
        res1.input.inflow = up_inflow.output
        res1.step(dt=manager.config.simulation_params.dt)
        
        # 4. Route flow through the channel
        channel.input.inflow = res1.input.release_outflow
        channel.step()
        
        # 5. Update reservoir 2
        res2.input.inflow = channel.output + down_inflow.output
        res2.step(dt=manager.config.simulation_params.dt)
        
        # 6. Calculate power for this step
        # Assume average head for simplicity
        power1 = calculate_power(res1.input.release_outflow, head=(180+200)/2)
        power2 = calculate_power(res2.input.release_outflow, head=(135+150)/2)
        total_power += (power1 + power2) * sim_config['simulation_params']['dt'] / 3600 # MWh
        
        history.append({
            'time': i,
            'res1_level': res1.level,
            'res2_level': res2.level,
            'res1_release': res1.input.release_outflow,
            'res2_release': res2.input.release_outflow
        })
        
    return pd.DataFrame(history), total_power

## 4. Evaluating Strategies and Finding the Best One

In [ ]:
# Define the different release strategies for 24 hours
steady_schedule = {
    'res_1': np.full(24, 200), # Match inflow
    'res_2': np.full(24, 250)  # Match its total inflow
}

peak_hours_schedule = {
    'res_1': np.array([150]*8 + [250]*8 + [150]*8), # Release more during hours 8-16
    'res_2': np.array([200]*8 + [300]*8 + [200]*8)
}

drain_down_schedule = {
    'res_1': np.full(24, 250),
    'res_2': np.full(24, 300)
}

strategies = {
    'Steady': steady_schedule,
    'Peak Hours': peak_hours_schedule,
    'Drain Down': drain_down_schedule
}

results = {}
power_results = {}

for name, schedule in strategies.items():
    print(f"--- Running Simulation for Strategy: {name} ---")
    df, total_power = run_simulation_with_schedule(config, schedule)
    results[name] = df
    power_results[name] = total_power
    print(f"Total Power Generated: {total_power:,.0f} MWh")

## 5. Visualizing the Results

In [ ]:
best_strategy_name = max(power_results, key=power_results.get)
print(f"\nBest strategy is: '{best_strategy_name}' with {power_results[best_strategy_name]:,.0f} MWh")

fig, axes = plt.subplots(3, 1, figsize=(15, 15), sharex=True)

# Plot Reservoir 1 Levels
for name, df in results.items():
    axes[0].plot(df['time'], df['res1_level'], label=name)
axes[0].set_title('Upstream Reservoir (res_1) Water Level')
axes[0].set_ylabel('Level (m)')
axes[0].grid(True)
axes[0].legend()

# Plot Reservoir 2 Levels
for name, df in results.items():
    axes[1].plot(df['time'], df['res2_level'], label=name)
axes[1].set_title('Downstream Reservoir (res_2) Water Level')
axes[1].set_ylabel('Level (m)')
axes[1].grid(True)
axes[1].legend()

# Plot Power Generation Results
axes[2].bar(power_results.keys(), power_results.values(), color=['gray', 'green', 'blue'])
axes[2].set_title('Total Power Generation by Strategy')
axes[2].set_ylabel('Total Power (MWh)')
axes[2].grid(axis='y')

plt.xlabel('Time (hours)')
plt.tight_layout()
plt.show()

### Conclusion

The results show that the **'Peak Hours'** strategy yields the most power. The 'Drain Down' strategy produces a lot of power but may not be sustainable and violates the water level constraints (which a real optimizer would prevent). The 'Steady' strategy is safe but leaves potential energy on the table.

This capstone notebook demonstrates how the components of the CHS SDK can be assembled to model, simulate, and analyze complex, system-level optimization problems. While our optimization was a simple heuristic search, it shows how the `SimulationManager` can be used as a core engine within a larger decision-making framework.